# Homework 5: Tree based classifiers

Summary: The purpose of this task is to gain experience and familiarity with using Tree based and KNN classification models. From there, we can put theory into practice by fully analyzing the results and data from the EPA. 

## Importing importing and implementing ML pipeline


In [189]:
# -----------------------------------
# Importing the necessary libraries
# -----------------------------------

import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# Libraries related to outlier detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import seaborn as sns
import warnings
import sklearn.metrics

import seaborn as sns
import warnings
from datetime import datetime
warnings.filterwarnings('ignore') 
sns.set(rc={'figure.figsize':(11,8)})
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:.2f}'.format



In [190]:
%cd '/Users/chandlersmith/Desktop/CS6140/HW4'
os.listdir()

# The first column is index: skipping that column
cars = pd.read_csv("cars_data_2023.csv")
print(cars.shape)

cars.head()

/Users/chandlersmith/Desktop/CS6140/HW4
(3377, 44)


,Model Year,Represented Test Veh Make,Represented Test Veh Model,Test Vehicle ID,Test Veh Configuration #,Test Veh Displacement (L),Vehicle Type,Rated Horsepower,# of Cylinders and Rotors,Tested Transmission Type Code,...,DT-Energy Economy Rating,Target Coef A (lbf),Target Coef B (lbf/mph),Target Coef C (lbf/mph**2),Set Coef A (lbf),Set Coef B (lbf/mph),Set Coef C (lbf/mph**2),Aftertreatment Device Cd,Aftertreatment Device Desc,Police - Emergency Vehicle?
0,2023,Aston Martin,DB11 V8,562TT5348,0,4.00,Car,503,8.00,SA,...,-7.71,40.94,0.02,0.03,11.26,0.09,0.03,TWC,Three-way catalyst,N
1,2023,Aston Martin,DB11 V8,562TT5348,0,4.00,Car,503,8.00,SA,...,-0.96,40.94,0.02,0.03,11.26,0.09,0.03,TWC,Three-way catalyst,N
2,2023,Aston Martin,DBS,7002PT7056,0,5.20,Car,715,12.00,SA,...,-0.58,40.94,0.02,0.03,6.81,0.08,0.02,TWC,Three-way catalyst,N
3,2023,Aston Martin,DBS,7002PT7056,0,5.20,Car,715,12.00,SA,...,-0.08,40.94,0.02,0.03,6.81,0.08,0.02,TWC,Three-way catalyst,N
4,2023,Aston Martin,DBX,8001PT8342,1,4.00,Both,550,8.00,A,...,-2.11,60.68,-0.33,0.04,-4.88,-0.53,0.04,TWC,Three-way catalyst,N


### Note befor advancing
Before we advance the basic ML pipeline: let's identidy and remove outliers in our dataset! 

## Outlier detection using Local Outlier Factor (LOF) method
- This method uses KNN

In [191]:
# -----------------------------------------------------------------------------
# Step 1
# Select a few important numerical features for outlier detection
# Make sure to avoid using Response variable (if one already exists)
# -----------------------------------------------------------------------------

num_cols = ['Rated Horsepower', 'Equivalent Test Weight (lbs.)','CO2 (g/mi)','RND_ADJ_FE']

# -----------------------------------------------------------------------------
# Step 2
# At this stage, either drop NAs or impute them with a value
# I have shown filling NAs with 0, as it seems approproate in this example            
# -----------------------------------------------------------------------------

X = cars[num_cols].fillna(0).values

# -----------------------------------------------------------------------------
# Step 3a
# fit the Local Outlier Factor model (based on KNN)
# Notice the contamination parameter to identify a certain proportion of outliers
# -----------------------------------------------------------------------------

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
# predict the labels for each data point (as Outlier or inlier)
y_pred_lof = lof.fit_predict(X)

# -----------------------------------------------------------------------------
# Step 3b
# fit the Isolation Forest outlier detection (based on decision trees)
# -----------------------------------------------------------------------------
iforest = IsolationForest(n_estimators=100,  contamination=0.05)
# predict the labels for each data point (as Outlier or inlier)
y_pred_if = iforest.fit_predict(X)


# -----------------------------------------------------------------------------
# Step 3c
# fit the robust covariance model (based on Mahalanobis distance)
# -----------------------------------------------------------------------------
rob_cov = EllipticEnvelope(contamination=0.05)
rob_cov.fit(X)

# predict the labels for each data point (as Outlier or inlier)
y_pred_rob = rob_cov.predict(X)

# -----------------------------------------------------------------------------
# Adding the newly created columns to the carsrotion table             
# -----------------------------------------------------------------------------
cars["y_pred_lof"] = y_pred_lof
cars["y_pred_if"] = y_pred_if
cars["y_pred_rob"] = y_pred_rob

# -----------------------------------------------------------------------------
# Converting them to a binary -1, 0. 
# Where -1 denotes outlier
# The purpose is to then add these columns and find out which rows were identified as outliers from multiple methods
# -----------------------------------------------------------------------------
cars["y_pred_lof_2"] = np.where(cars["y_pred_lof"]<0, -1, 0)
cars["y_pred_if_2"] = np.where(cars["y_pred_if"]<0, -1, 0)
cars["y_pred_rob_2"] = np.where(cars["y_pred_rob"]<0, -1, 0)




## Summing the outlier status 

In [192]:
cars.iloc[:,-3:]
pd.crosstab(cars["y_pred_if"], cars["y_pred_rob"] )

cars["all_out"] = cars.loc[:,["y_pred_if_2","y_pred_rob_2","y_pred_lof_2"]].sum(axis = 1)
cars["all_out"].value_counts()

# -----------------------------------------------------------------------------
# List of cars identified as outliers based by at least two methods
# -----------------------------------------------------------------------------
cars[cars["all_out"]<-1]
# Double check the data in csv
#outliers = cars[cars["all_out"]<-1]
#outliers.to_csv( "outliers.csv", index=False)

,Model Year,Represented Test Veh Make,Represented Test Veh Model,Test Vehicle ID,Test Veh Configuration #,Test Veh Displacement (L),Vehicle Type,Rated Horsepower,# of Cylinders and Rotors,Tested Transmission Type Code,...,Aftertreatment Device Cd,Aftertreatment Device Desc,Police - Emergency Vehicle?,y_pred_lof,y_pred_if,y_pred_rob,y_pred_lof_2,y_pred_if_2,y_pred_rob_2,all_out
210,2023,BMW,MINI COOPER SE HARDTOP 2 DOOR,2N13701,0,0.00,Car,181,NaN,A,...,NaN,NaN,N,-1,-1,-1,-1,-1,-1,-3
211,2023,BMW,MINI COOPER SE HARDTOP 2 DOOR,2N13701,0,0.00,Car,181,NaN,A,...,NaN,NaN,N,-1,-1,-1,-1,-1,-1,-3
267,2023,BMW,X5 xDrive45e,LE48294,2,3.00,Both,282,6.00,SA,...,TWC,Three-way catalyst,N,-1,-1,-1,-1,-1,-1,-3
269,2023,BMW,X5 xDrive45e,LE48294,2,3.00,Both,282,6.00,SA,...,TWC,Three-way catalyst,N,-1,-1,-1,-1,-1,-1,-3
379,2023,MINI,Mini Cooper SE Countryman ALL4,3F94106,2,1.50,Car,134,3.00,SA,...,TWC,Three-way catalyst,N,-1,1,-1,-1,0,-1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3222,2023,BUGATTI,Bugatti Pur Sport,BG744-PT50 CHIRON R,0,7.99,Car,1500,16.00,AMS,...,TWC,Three-way catalyst,N,-1,-1,1,-1,-1,0,-2
3297,2023,Volkswagen,ID.4 AWD Pro S,VW316630121,1,0.00,Truck,295,NaN,A,...,NaN,NaN,N,1,-1,-1,0,-1,-1,-2
3299,2023,Volkswagen,ID.4 Pro S,VW316630184,0,0.00,Truck,201,NaN,A,...,NaN,NaN,N,1,-1,-1,0,-1,-1,-2
3301,2023,Volkswagen,ID.4 Pro S,VW316630184,0,0.00,Truck,201,NaN,A,...,NaN,NaN,N,1,-1,-1,0,-1,-1,-2


Outlier Analysis: So let's take a step back, what did we just do. Essentially, we used Mahalanobis distance and KNN to identify which cars were outliers! The result was a list of 113 rows that we should remove prior to conducting the rest of the analysis. Now lets continue with our ML Pipeline. It is a bit easier to review the data in the csv file so feel free to uncomment the code and check yourself. It is very clear that many of the horsepower outliers were extremely low, down at 37. The high end of horsepower was 1500, far above the expected mean. Additionally, test weight was as high as 7000 and as low as 2375. I imagine most outliers were high though. 3rd Metric was C02, and, as expected, were hybrid or electric cars. And finally, RND_ADJ_FE had drastic outliers from electric and highbrid vehicals, similar to C02 emissions. Removing the electric cards from the dataset seemed a wise next step to take.

In [193]:
from sklearn.preprocessing import LabelEncoder

# Drop columns with lots of gaps to preserve columns we really care about. 
# cars = cars.drop(["DT-Inertia Work Ratio Rating", "DT-Absolute Speed Change Ratg", "DT-Energy Economy Rating", "N2O (g/mi)", "CH4 (g/mi)",
  #         "PM (g/mi)", "NOx (g/mi)", "THC (g/mi)"], inplace=True)

# remove outliers from df
#cars = cars.loc(cars["all_out"]<-1)
cars.drop('CO2 (g/mi)', axis=1)
cars = cars.dropna()
cars.head()


#standardize all numerical (except for the response)
for i in cars:
    if cars[i].dtypes == 'number' and i != 'RND_ADJ_FE':
        cars[i] = StandardScaler.fit_transform(cars[i])


#Encode all categorical
for i in cars:
    if cars[i].dtype == "object":
        encode = LabelEncoder()
        cars[i] = encode.fit_transform(cars[i])

cars.to_csv("test.csv", index=False)        



Now that we have our dataset, let's move into SGDRegressor and SGDClassifier

## SGD Regressor

In [194]:
# --------------------------------------------------------------------------------
# # Separating the features and target
# Notice no attention is paid to Test-Train separation. Consider that step as an 
# intergal part of ML pipeline, and should not be skipped
# --------------------------------------------------------------------------------

X = cars.iloc[:, cars.columns != 'RND_ADJ_FE']
y = cars["RND_ADJ_FE"] 

# # ----------------------------------------------------------------------
# # # # Feature Standardization
# # This is a required step, as recommended by sklearn
# # ----------------------------------------------------------------------
scaler = StandardScaler()
X = scaler.fit_transform(X)

# --------------------------
# # # Training the model
# --------------------------
model = SGDRegressor(max_iter=100, 
                     tol = 0.0001,
                     early_stopping=False, warm_start=False,
                     n_iter_no_change = 5)
model.fit(X,y)
    
# -------
# Predict    
# -------
y_pred = model.predict(X)

# ----------------------
# Evaluating using MSE
# Using sklearn's built in function
# ----------------------

from sklearn.metrics import mean_squared_error
print(f"MSE is {mean_squared_error(y, y_pred)}")

# --------------------------------------------
# Alternatively writing the formula
# --------------------------------------------

mse = np.mean((y - y_pred)**2)




print(f"Number of adjustments in weights = {model.t_}, Coefficients are {model.coef_}, Number of iterations=  {model.n_iter_} and the R2 score is {model.score(X,y)}")
print("\nNote that the coefficients are for the standardized data and not on the original scale\n")
    
   

MSE is 5.633129421620811
Number of adjustments in weights = 16567.0, Coefficients are [ 0.          0.53353301 -0.1402651  -0.25657009  0.09624572 -0.17276312
 -0.20715     0.16835158  0.51132895 -0.72821943  0.62019995 -0.07708415
 -0.25723064  0.22637532  0.35330731 -0.21324003 -0.22333547  0.24540411
  0.05487658  1.68907638 -0.95779756  0.06749147 -0.59223205  0.5667204
 -0.49099135 -7.66586099 -0.13103077  0.0283798  -0.29794487  0.56636177
  0.          0.85212137  0.20701999 -0.75539215 -0.37925249 -0.60540583
 -0.76609825 -0.16807647 -0.21505264  0.25509175 -0.2061335  -0.01358851
  0.         -0.03718356 -0.38044166 -0.37594144 -0.03718356 -0.38044166
 -0.37594144 -0.33126004], Number of iterations=  33 and the R2 score is 0.9125853181254651

Note that the coefficients are for the standardized data and not on the original scale



Results from SGD Regressor: In SGD regression analysis, the algorithm randomly selects a batch from the training data for each iteration, computes the gradient of the loss function, and updates the model. The results will change and vary because we are working with training data, which naturally will return slightly different results. The overall number of iterations was 25, and the R2 score is 0.9076. This is a very high R2 value even after removing the c02 predictor but makes sense because we retiained a high number of features. If there were more data, I would like want to reduce the number of features and remove the lease influential ones. The regressor model, with a R2 score of 0.9076, could therefore be expected to explain around 90% of the miles per gallon (target) variable provided the input features. This is a high performant model and there isn't much we can do to improve the model other than increase data or put back in the C02 feature. More to come on the comparison for the classifier. 



## SGD Classifier

In [195]:
# ----------------------------------------
#importing necessary libraries
# ----------------------------------------

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics

X = cars.iloc[:, cars.columns != 'RND_ADJ_FE']
y = cars["RND_ADJ_FE"] 

# High when MPG is 30, else low
y_cat = np.where(y>29, 0,1)

# # # ----------------------------------------------------------------------
# # # # # Feature Standardization
# # # This is a required step, as recommended by sklearn
# # # ----------------------------------------------------------------------
scaler = StandardScaler()
X = scaler.fit_transform(X)


#splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = 0.2, random_state=42)

#creating and fitting the SGDClassifier
clf = SGDClassifier(max_iter=1000, tol=1e-3, loss = 'log')
clf.fit(X_train, y_train)

#predicting the test set results
y_pred = clf.predict(X_test)

#calculating the accuracy of the model
accuracy = clf.score(X_test, y_test)

print("Accuracy: {:.2f}".format(accuracy))

Accuracy: 0.94


In [196]:
# help(sklearn.metrics)
# ------------------------------
# Various classification metrics
# ------------------------------

print(f"Overall accuracy is {sklearn.metrics.accuracy_score(y_pred, y_test)}")
print(f"Precision or TP/(TP + FP) is {sklearn.metrics.precision_score(y_pred, y_test)}")
print(f"Recall or TP / (TP + FN) is {sklearn.metrics.recall_score(y_pred, y_test)}")
print(f"F1-score or 2*Precision*Recall / (Precision + Recall)  is {sklearn.metrics.f1_score(y_pred, y_test)}")
print(f"Confusion Matrix \n{sklearn.metrics.confusion_matrix(y_pred, y_test)}")


Overall accuracy is 0.9405940594059405
Precision or TP/(TP + FP) is 0.9552238805970149
Recall or TP / (TP + FN) is 0.9552238805970149
F1-score or 2*Precision*Recall / (Precision + Recall)  is 0.9552238805970149
Confusion Matrix 
[[31  3]
 [ 3 64]]


In [197]:
#predicting the test set results
y_pred = clf.predict(X_test)

# # predict the decision scores for the data
decision_scores = clf.decision_function(X_test)

# # change the decision threshold to 0.6
y_pred_new = (decision_scores > 0.6).astype(int)

# # print the accuracy of the new predictions
print("Accuracy with threshold of 0.6:", sum(y_pred_new == y_test) / len(y_test))

Accuracy with threshold of 0.6: 0.9405940594059405


Results from SGD Classifier: This is test which basically says, can this model predict the right result, yes or no. It turns out, this model is pretty good, however, not quite as accurate as the regression. Accuracy with threshold of 0.6: 0.8811881188118812 means that when using a threshold of 0.6, the model was able to classify the row correctly about 88% of the time. This is within a couple of percentage points for the regressor models. One action we could take to improve the model is potentially up the decision threshold, which might improve the model. Additionally, adding the C02 feature back in would help. Both the vlassifier and the regression model were highly performant and there is not much to split the two in terms of accuracy. They do tell us slightly different things, hovever, and it is probably good to use both to verify one another. 

## KNN Regression

In [198]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score


X = cars.iloc[:, cars.columns != 'RND_ADJ_FE']
y = cars["RND_ADJ_FE"] 
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [199]:
# ---------------------------------------------
# Split data into training and testing sets
# ---------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ---------------------------------------------
# Train a KNN regressor with 5 neighbors
# Choice of neighbors is hyperparameter
# ---------------------------------------------
knn_reg = KNeighborsRegressor(n_neighbors=5, leaf_size=10)
knn_reg.fit(X_train, y_train)

# ---------------------------------------------
# Make predictions on the testing set
# ---------------------------------------------
y_pred = knn_reg.predict(X_test)

# ---------------------------------------------
# Evaluate the MSE and R-squared values 
# ---------------------------------------------
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse}")
print(f"R-sq value is {r2_score(y_test, y_pred)}")

Mean squared error: 13.375584105960263
R-sq value is 0.7793169367614097


KNN Regressor results: The KNN regressor is designed to predict a target based on the nearest neighbors. The MSE, in this case 13.42, explains the distance between the predicted values and the actual target values. This is a pretty high different given the context of the model and that is verified by the model's R-sq value sitting at 0.77, far lower than the SGD regression R-sq value calculated above. Some things we could do to improve the model are to tune the neighbor count and the leaf size. Perhaps the number of neighbors or leaf size could be impacting the accuracy of the model, and this is going to specific to the data set. As a check I made the neighbors 10, and that lowered the models accuracy slightly and then tune the leaf size to 5 which drastically lowered the R-sq value. There is some wiggle room here and it would be worth trying this a few times. 

## KNN Classifier

In [200]:
# ----------------------------------------
#Importing data and creating a binary response
# ----------------------------------------

X = cars.iloc[:, cars.columns != 'RND_ADJ_FE']
y = cars["RND_ADJ_FE"] 

# MPH > 30, else Low
y_cat = np.where(y<29, 0,1)

# Feature Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)

# KNN classifier with 5 neighbors
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn_clf.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = knn_clf.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9207920792079208


Summary of KNN Regressor Results: A KNN classifier accuracy score is essentially the % with which the model can correctly identify the classified result. In this case, it was fuel consumption high or low. Here, we note that the KNN classifier performed lower when compared to the SGD model, however, it still performed pretty well. Some actions that can be taken to improve the model are to adjust the n_neighbors variable and rerun the model. Additionally, we could play around with the MPG line, 30 might not be the best high low measure here. 

Sumamry of all 4
In conclusion, both SGD models were able to predict with a higher level of confidence the target variable when compared to the KNN models. The regression models predict the target provided the features while the classification models' accuracy score shares the accuracy of the model predicting high low on miles per gallon. For SGD, playing with the batch size, training data, and rerunning the model can be strong hyperparameters to tune. For KNN, the number of neighbors, leaf size, and training data are tunable hyperparameters. Overall, SGD regression and classigication outperformed KNN models, however, both performaed well. In terms of data anomolies, electric and hybrid vehicles really through the models through a loop. In the future, I would split the models into three catagories: Hybrid, Electric, and Petrol vehicals and this would likely yield more accurate petrol KNN and SGD models. We probably don't have enough Electric or Hybrid vehicals on the market to run a full model, but in a few years, that will likely change. 


# HW5 Starts here! The above is HW 4


Question: Did the EPA claim that pollutants levals generally decreased over time?

## Data Cleaning
Now that we have confirmed 41 features successfully read in, we need to do the following. 
Remove all vehicles with mpg > 120 and drop commmonly empty columns. 


In [201]:
%cd '/Users/chandlersmith/Desktop/CS6140/HW5'
os.listdir()

# The first column is index: skipping that column
EPAcars = pd.read_csv("cars_10_16_23.csv")
print(EPAcars.shape)

#EPAcars.head()

dropCols = ['Aftertreatment Device Desc','# of Cylinders and Rotors', 
                        'Aftertreatment Device Cd', 'THC (g/mi)', 'CH4 (g/mi)', 
                        'NOx (g/mi)', 'PM (g/mi)', 'N2O (g/mi)']
cars = EPAcars.drop(dropCols, axis=1)
cars.dropna()
print(cars.head())


/Users/chandlersmith/Desktop/CS6140/HW5
(11304, 42)
   Unnamed: 0 Represented Test Veh Make Transmission Overdrive Desc  \
0           0              Aston Martin          Top gear ratio < 1   
1           1              Aston Martin          Top gear ratio < 1   
2           2              Aston Martin          Top gear ratio < 1   
3           3              Aston Martin          Top gear ratio < 1   
4           4              Aston Martin          Top gear ratio < 1   

   Equivalent Test Weight (lbs.) Represented Test Veh Model  \
0                           4500                    DB11 V8   
1                           4500                    DB11 V8   
2                           4500                        DBS   
3                           4500                        DBS   
4                           5500                        DBX   

  Drive System Description  Test Veh Displacement (L)  \
0      2-Wheel Drive, Rear                       4.00   
1      2-Wheel Drive, Rear  

In [202]:
# -----------------------------------------------------------------------------
# Step 1
# Select a few important numerical features for outlier detection
# Make sure to avoid using Response variable (if one already exists)
# -----------------------------------------------------------------------------

num_cols = ['Rated Horsepower', 'Equivalent Test Weight (lbs.)','CO2 (g/mi)','RND_ADJ_FE']

# -----------------------------------------------------------------------------
# Step 2
# At this stage, either drop NAs or impute them with a value
# I have shown filling NAs with 0, as it seems approproate in this example            
# -----------------------------------------------------------------------------

X = cars[num_cols].fillna(0).values

# -----------------------------------------------------------------------------
# Step 3a
# fit the Local Outlier Factor model (based on KNN)
# Notice the contamination parameter to identify a certain proportion of outliers
# -----------------------------------------------------------------------------

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
# predict the labels for each data point (as Outlier or inlier)
y_pred_lof = lof.fit_predict(X)

# -----------------------------------------------------------------------------
# Step 3b
# fit the Isolation Forest outlier detection (based on decision trees)
# -----------------------------------------------------------------------------
iforest = IsolationForest(n_estimators=100,  contamination=0.05)
# predict the labels for each data point (as Outlier or inlier)
y_pred_if = iforest.fit_predict(X)


# -----------------------------------------------------------------------------
# Step 3c
# fit the robust covariance model (based on Mahalanobis distance)
# -----------------------------------------------------------------------------
rob_cov = EllipticEnvelope(contamination=0.05)
rob_cov.fit(X)

# predict the labels for each data point (as Outlier or inlier)
y_pred_rob = rob_cov.predict(X)

# -----------------------------------------------------------------------------
# Adding the newly created columns to the carsrotion table             
# -----------------------------------------------------------------------------
cars["y_pred_lof"] = y_pred_lof
cars["y_pred_if"] = y_pred_if
cars["y_pred_rob"] = y_pred_rob

cars.iloc[:,-3:]
pd.crosstab(cars["y_pred_if"], cars["y_pred_rob"] )

cars["all_out"] = cars.loc[:,["y_pred_if","y_pred_rob","y_pred_lof"]].sum(axis = 1)
cars["all_out"].value_counts()

# -----------------------------------------------------------------------------
# List of cars identified as outliers based by at least two methods
# -----------------------------------------------------------------------------
cars[cars["all_out"]<-1]

,Unnamed: 0,Represented Test Veh Make,Transmission Overdrive Desc,Equivalent Test Weight (lbs.),Represented Test Veh Model,Drive System Description,Test Veh Displacement (L),Shift Indicator Light Use Desc,N/V Ratio,Rated Horsepower,...,RND_ADJ_FE,CO2 (g/mi),Tested Transmission Type Code,Vehicle Type,Set Coef B (lbf/mph),Target Coef C (lbf/mph**2),y_pred_lof,y_pred_if,y_pred_rob,all_out
663,663,Ford,Top gear ratio < 1,7000,F-150 Lightning,4-Wheel Drive,0.00,Not eqipped,101.10,420,...,111.20,NaN,OT,Truck,-0.01,0.03,-1,-1,-1,-3
665,665,Ford,Top gear ratio < 1,7000,F-150 Lightning Platinum,Part-time 4-Wheel Drive,0.00,Not eqipped,101.60,420,...,104.30,NaN,OT,Truck,-0.06,0.04,-1,-1,-1,-3
871,871,CADILLAC,No gear ratio < 1,6000,LYRIQ EV,4-Wheel Drive,0.00,Not eqipped,132.40,342,...,141.00,NaN,A,Truck,0.21,0.02,-1,-1,-1,-3
873,873,CADILLAC,No gear ratio < 1,6000,LYRIQ EV,"2-Wheel Drive, Rear",0.00,Not eqipped,132.40,342,...,136.00,NaN,A,Truck,0.21,0.02,-1,-1,-1,-3
1190,1190,CHEVROLET,Top gear ratio < 1,3500,TRAILBLAZER AWD,"2-Wheel Drive, Front",1.30,Not eqipped,24.70,155,...,10000.00,189.50,A,Car,0.15,0.02,-1,-1,-1,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053,4399,AUDI,Top gear ration < 1,4250,Audi A6,"2-Wheel Drive, Front",1.98,"Equipped, not shifted by SIL",22.40,252,...,10000.00,178.00,SA,Car,-0.10,0.02,-1,-1,-1,-3
11054,4400,AUDI,Top gear ration < 1,4250,Audi A6,"2-Wheel Drive, Front",1.98,"Equipped, not shifted by SIL",22.40,252,...,10000.00,271.00,SA,Car,-0.10,0.02,-1,-1,-1,-3
11055,4401,AUDI,Top gear ration < 1,4250,Audi A6,"2-Wheel Drive, Front",1.98,"Equipped, not shifted by SIL",26.90,252,...,10000.00,305.00,SA,Car,0.22,0.02,-1,-1,-1,-3
11056,4402,AUDI,Top gear ration < 1,4250,Audi A6,"2-Wheel Drive, Front",1.98,"Equipped, not shifted by SIL",26.90,252,...,10000.00,320.00,SA,Car,0.22,0.02,-1,-1,-1,-3


# CO2 Model

In [203]:
#standardize all numerical (except for the response)
for i in cars:
    if cars[i].dtypes == 'number' and i != 'CO2 (g/mi)': # and i != 'CO (g/mi)'
        cars[i] = StandardScaler.fit_transform(cars[i])


#Encode all categorical
for i in cars:
    if cars[i].dtype == "object":
        encode = LabelEncoder()
        cars[i] = encode.fit_transform(cars[i])

cars.to_csv("test.csv", index=False)        

## Convert CO2 into binary
# ------------------------------
# Response variable
#	CO2 > 600 = High, else Other
#	CO > 1 = High, else Other

# ------------------------------
cars['CO2 (g/mi)'] = cars['CO2 (g/mi)'].apply(lambda x: 1 if x > 600 else 0)
cars.dropna()

# Drop all rows where MPH is > 120
cars = cars[cars['RND_ADJ_FE'] <= 120.0]


# Make cars = all but CO2
X = cars.drop(['CO2 (g/mi)', 'CO (g/mi)'], axis = 1)
y = cars["CO2 (g/mi)"] 
# Unsure how to incorperate 'CO (g/mi)'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(X_train.head())


### Logistic Regression

In [204]:
LR = sklearn.linear_model.LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.9859791949344188


## Tree Based Models

### Decision Tree Classifier

In [205]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

# ---------------------------------------------------------------
# Train a decision tree classifier on the balanced dataset
# ---------------------------------------------------------------

clf = DecisionTreeClassifier(criterion='gini',
                             min_samples_split=5,
                             min_samples_leaf=5)
clf.fit(X_train, y_train)

# --------------------------------------------
# Evaluate the classifier on the testing set
# --------------------------------------------
y_pred_dt = clf.predict(X_test)
print(classification_report(y_test, y_pred_dt))

confusion_matrix(y_test, y_pred_dt)

# Confusion Matrix
pd.crosstab(y_pred_dt, y_test, rownames =['y_pred_dt'], colnames = ['y_test'] )


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2158
           1       0.98      0.94      0.96        53

    accuracy                           1.00      2211
   macro avg       0.99      0.97      0.98      2211
weighted avg       1.00      1.00      1.00      2211



y_test,0,1
y_pred_dt,,
0,2157,3
1,1,50


### Random Forest

In [206]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ------------------
# Define the RF model
# ------------------
rf = RandomForestClassifier(n_estimators=1000,
                            criterion='entropy',
                            min_samples_split=7,
                            min_samples_leaf=7,
                            random_state=100)

# ------------------
# Train the models
# ------------------
rf.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_rf = rf.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rf))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf))
print('Random Forest Recall:', recall_score(y_test, y_pred_rf))
print('Random Forest F1 Score:', f1_score(y_test, y_pred_rf))

print(classification_report(y_test, y_pred_rf))

# Confusion Matrix
pd.crosstab(y_pred_rf, y_test, rownames =['y_pred_rf'], colnames = ['y_test'] )

Random Forest Accuracy: 0.9990954319312528
Random Forest Precision: 0.9811320754716981
Random Forest Recall: 0.9811320754716981
Random Forest F1 Score: 0.9811320754716981
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2158
           1       0.98      0.98      0.98        53

    accuracy                           1.00      2211
   macro avg       0.99      0.99      0.99      2211
weighted avg       1.00      1.00      1.00      2211



y_test,0,1
y_pred_rf,,
0,2157,1
1,1,52


### Gradient Boosted Tree

In [207]:
from sklearn.ensemble import GradientBoostingClassifier

# ------------------
# Define the GB model
# ------------------
gb = GradientBoostingClassifier(n_estimators=500,
                            min_samples_split=10,
                            learning_rate=.5,
                            min_samples_leaf=5,
                            random_state=100)

# ------------------
# Train the model
# ------------------
gb.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_gb = gb.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------

print('Gradient Boosting Accuracy:', accuracy_score(y_test, y_pred_gb))
print('Gradient Boosting Precision:', precision_score(y_test, y_pred_gb))
print('Gradient Boosting Recall:', recall_score(y_test, y_pred_gb))
print('Gradient Boosting F1 Score:', f1_score(y_test, y_pred_gb))

print(classification_report(y_test, y_pred_gb))
# Confusion Matrix
pd.crosstab(y_pred_gb, y_test, rownames =['y_pred_gb'], colnames = ['y_test'] )

Gradient Boosting Accuracy: 0.9981908638625057
Gradient Boosting Precision: 0.9454545454545454
Gradient Boosting Recall: 0.9811320754716981
Gradient Boosting F1 Score: 0.9629629629629629
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2158
           1       0.95      0.98      0.96        53

    accuracy                           1.00      2211
   macro avg       0.97      0.99      0.98      2211
weighted avg       1.00      1.00      1.00      2211



y_test,0,1
y_pred_gb,,
0,2155,1
1,3,52


### XGBoost

In [208]:
import xgboost as xgb

# ------------------
# Define the XGB model
# ------------------
xgb = xgb.XGBClassifier(n_estimators=100,
                        max_depth = 3,
                        eta= 0.1,
                        min_child_weight = 5,
                        random_state=100)

# ------------------
# Train the model
# ------------------
xgb.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_xgb = xgb.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------

print('XGB:', accuracy_score(y_test, y_pred_xgb))
print('XGB:', precision_score(y_test, y_pred_xgb))
print('XGB:', recall_score(y_test, y_pred_xgb))
print('XGB:', f1_score(y_test, y_pred_xgb))

print(classification_report(y_test, y_pred_xgb))
# Confusion Matrix
pd.crosstab(y_pred_xgb, y_test, rownames =['y_pred_xgb'], colnames = ['y_test'] )

XGB: 0.9986431478968792
XGB: 0.9807692307692307
XGB: 0.9622641509433962
XGB: 0.9714285714285713
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2158
           1       0.98      0.96      0.97        53

    accuracy                           1.00      2211
   macro avg       0.99      0.98      0.99      2211
weighted avg       1.00      1.00      1.00      2211



y_test,0,1
y_pred_xgb,,
0,2157,2
1,1,51


# CO Model

In [209]:
# Reload the data 
%cd '/Users/chandlersmith/Desktop/CS6140/HW5'
os.listdir()

# The first column is index: skipping that column
EPAcars = pd.read_csv("cars_10_16_23.csv")
print(EPAcars.shape)

dropCols = ['Aftertreatment Device Desc','# of Cylinders and Rotors', 
                        'Aftertreatment Device Cd', 'THC (g/mi)', 'CH4 (g/mi)', 
                        'NOx (g/mi)', 'PM (g/mi)', 'N2O (g/mi)']
cars = EPAcars.drop(dropCols, axis=1)
cars.dropna()


#standardize all numerical (except for the response)
for i in cars:
    if cars[i].dtypes == 'number' and i != 'CO (g/mi)': # and i != 'CO (g/mi)'
        cars[i] = StandardScaler.fit_transform(cars[i])


#Encode all categorical
for i in cars:
    if cars[i].dtype == "object":
        encode = LabelEncoder()
        cars[i] = encode.fit_transform(cars[i])

cars.to_csv("test.csv", index=False)        

## Convert CO2 into binary
# ------------------------------
# Response variable
#	CO2 > 600 = High, else Other
#	CO > 1 = High, else Other

# ------------------------------
cars['CO (g/mi)'] = cars['CO (g/mi)'].apply(lambda x: 1 if x > 1 else 0)
cars.dropna()

# Drop all rows where MPH is > 120
cars = cars[cars['RND_ADJ_FE'] <= 120.0]


# Make cars = all but CO2
X = cars.drop(['CO2 (g/mi)', 'CO (g/mi)'], axis = 1)
y = cars["CO (g/mi)"] 
# Unsure how to incorperate 'CO (g/mi)'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/Users/chandlersmith/Desktop/CS6140/HW5
(11304, 42)


### Logistic Regression

In [210]:
LR = sklearn.linear_model.LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.9393939393939394


## Tree Based Models

### Decision Tree Classifier

In [211]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

# ---------------------------------------------------------------
# Train a decision tree classifier on the balanced dataset
# ---------------------------------------------------------------

clf = DecisionTreeClassifier(criterion='gini',
                             min_samples_split=5,
                             min_samples_leaf=5)
clf.fit(X_train, y_train)

# --------------------------------------------
# Evaluate the classifier on the testing set
# --------------------------------------------
y_pred_dt = clf.predict(X_test)
print(classification_report(y_test, y_pred_dt))

confusion_matrix(y_test, y_pred_dt)

# Confusion Matrix
pd.crosstab(y_pred_dt, y_test, rownames =['y_pred_dt'], colnames = ['y_test'] )

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      2076
           1       0.63      0.51      0.57       135

    accuracy                           0.95      2211
   macro avg       0.80      0.75      0.77      2211
weighted avg       0.95      0.95      0.95      2211



y_test,0,1
y_pred_dt,,
0,2036,66
1,40,69


### Random Forest

In [212]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ------------------
# Define the RF model
# ------------------
rf = RandomForestClassifier(n_estimators=1000,
                            criterion='entropy',
                            min_samples_split=5,
                            min_samples_leaf=5,
                            random_state=100)

# ------------------
# Train the models
# ------------------
rf.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_rf = rf.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred_rf))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf))
print('Random Forest Recall:', recall_score(y_test, y_pred_rf))
print('Random Forest F1 Score:', f1_score(y_test, y_pred_rf))

print(classification_report(y_test, y_pred_rf))

# Confusion Matrix
pd.crosstab(y_pred_rf, y_test, rownames =['y_pred_rf'], colnames = ['y_test'] )

Random Forest Accuracy: 0.9556761646313885
Random Forest Precision: 0.7846153846153846
Random Forest Recall: 0.37777777777777777
Random Forest F1 Score: 0.5100000000000001
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      2076
           1       0.78      0.38      0.51       135

    accuracy                           0.96      2211
   macro avg       0.87      0.69      0.74      2211
weighted avg       0.95      0.96      0.95      2211



y_test,0,1
y_pred_rf,,
0,2062,84
1,14,51


### Gradient Boosted Tree

In [213]:
from sklearn.ensemble import GradientBoostingClassifier

# ------------------
# Define the GB model
# ------------------
gb = GradientBoostingClassifier(n_estimators=500,
                            min_samples_split=5,
                            learning_rate=0.5,
                            min_samples_leaf=5,
                            random_state=100)

# ------------------
# Train the model
# ------------------
gb.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_gb = gb.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------

print('Gradient Boosting Accuracy:', accuracy_score(y_test, y_pred_gb))
print('Gradient Boosting Precision:', precision_score(y_test, y_pred_gb))
print('Gradient Boosting Recall:', recall_score(y_test, y_pred_gb))
print('Gradient Boosting F1 Score:', f1_score(y_test, y_pred_gb))

print(classification_report(y_test, y_pred_gb))
# Confusion Matrix
pd.crosstab(y_pred_gb, y_test, rownames =['y_pred_gb'], colnames = ['y_test'] )

Gradient Boosting Accuracy: 0.9579375848032564
Gradient Boosting Precision: 0.6810344827586207
Gradient Boosting Recall: 0.5851851851851851
Gradient Boosting F1 Score: 0.6294820717131474
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2076
           1       0.68      0.59      0.63       135

    accuracy                           0.96      2211
   macro avg       0.83      0.78      0.80      2211
weighted avg       0.96      0.96      0.96      2211



y_test,0,1
y_pred_gb,,
0,2039,56
1,37,79


### XGBoost

In [214]:
import xgboost as xgb

# ------------------
# Define the XGB model
# ------------------
xgb = xgb.XGBClassifier(n_estimators=100,
                        max_depth = 10,
                        eta= 0.05,
                        min_child_weight = 5,
                        random_state=100)

# ------------------
# Train the model
# ------------------
xgb.fit(X_train, y_train)

# ------------------------------------
# Make predictions on the test set
# ------------------------------------
y_pred_xgb = xgb.predict(X_test)

# ------------------------------------------------------------------------
# Evaluate the model using accuracy, precision, recall, and F1 score
# ------------------------------------------------------------------------

print('XGB:', accuracy_score(y_test, y_pred_xgb))
print('XGB:', precision_score(y_test, y_pred_xgb))
print('XGB:', recall_score(y_test, y_pred_xgb))
print('XGB:', f1_score(y_test, y_pred_xgb))

print(classification_report(y_test, y_pred_xgb))
# Confusion Matrix
pd.crosstab(y_pred_xgb, y_test, rownames =['y_pred_xgb'], colnames = ['y_test'] )

XGB: 0.9620081411126187
XGB: 0.7865168539325843
XGB: 0.5185185185185185
XGB: 0.625
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2076
           1       0.79      0.52      0.62       135

    accuracy                           0.96      2211
   macro avg       0.88      0.75      0.80      2211
weighted avg       0.96      0.96      0.96      2211



y_test,0,1
y_pred_xgb,,
0,2057,65
1,19,70


## Comparison and Analysis of CO2 vs CO models



#### Summary 
At the beginning of both data sets I conducted clearning and outlier analysis which involved removing several columns and all rows with MPG above 120 (likely indicating the vehical was electric or hybrid). In general, this was done as instructed but in the broader context of the EPA's analysis I would prefer to include those vehicals. I also removed na's and encoded categorical data. For each model, I removed the other indicator variab (eg. CO was removed in the CO2 model). Hyperparameter tuning is shown throughout. 

Overall, I thought the models were far too accurate for CO2 but I review the Xtrain and Test data and I could not pinpoint why. Perhaps it has to do with Set Coef B (lbf/mph)  Target Coef C (lbf/mph**2) columns, but I am not sure. I tested without and it didn't make much difference. A more likely story is that the encoding binary classifier is extremely devicisive. 

##### Logistic Regression:
CO2: 0.98
CO: 0.93

##### Decision Tree Classifier:
Split and leaf - tried 3, 5, and 7. Overall, increasing the split and leaves reduced overall accuracy. In the CO model, reducing the min samples leaf to 1 brought the precision down a little. 
Macro avg: (CO2 / CO)
    Precision   0.98/0.81
    Recall      0.97/0.76
    f1-score    0.98/0.78


##### Random Forest
Split and leaf - tried 3, 5, and 7. Increasing split and leaf again redued accuracy when considering a macro average. Increasing the min samples leaf in the CO2 model seemingly increased its runtime and did not improve the model in all three categories. 
Macro avg
    Precision   0.97/0.87
    Recall      0.99/0.69
    f1-score    0.98/0.74

##### Gradient Boosted Tree
Split and leaf - tried 3, 5, and 7. I dropped the n_estimators, agjusted the min_samples_split, and also changed the learning rate and found little variation. A learning rate of 1 broke all but the accuracy score of a model. I expected a Learning rate of 0.1 to show some difference but it did not in the CO2 model. In the CO model, decreasing the learning rate to 0.1 improved the models precision byt not it's recall or F1 Score. This is surprising as I would expect these 4 numbers to move together. 
Macro avg
    Accuracy    0.99/0.95
    Precision   0.95/ 0.68
    Recall      0.98/0.58
    f1-score    0.96/0.63


##### XGBoost
Increasing the max depth to 1 generated a recall_score of 1. This means at this depth it coudld bascially perfectly classify the data as true positives were 1! Adjusting the learning rate(eta) to 0.1 for the CO2 model did little to impact the model. In the CO model, increasing the eta had the biggest impact on the models accuracy. A higher learning rate can be better to an extent because it allows the model to learn more quickly and converge to a good solution faster. With a higher learning rate, the model updates its weights more aggressively, leading to larger changes in the decision boundaries of the model.
Macro avg
    Accuracy    0.99/0.96
    Precision   0.96/0.79
    Recall      0.94/0.51
    f1-score    0.95/0.63


## Test EPA's Claim that the pollutants' levels decrease over time

Methodology: Confirm basic stats of mean, median, etc, are down from 2010 -> 2016 and 2016-> 2023.



In [215]:
# Reload the data 
%cd '/Users/chandlersmith/Desktop/CS6140/HW5'
os.listdir()

# The first column is index: skipping that column
EPAcars = pd.read_csv("cars_10_16_23.csv")
print(EPAcars.shape)

dropCols = ['Aftertreatment Device Desc','# of Cylinders and Rotors', 
                        'Aftertreatment Device Cd', 'THC (g/mi)', 'CH4 (g/mi)', 
                        'NOx (g/mi)', 'PM (g/mi)', 'N2O (g/mi)']
cars = EPAcars.drop(dropCols, axis=1)
cars.dropna()
cars = cars[cars['RND_ADJ_FE'] <= 120.0]
# define pollutant levels as CO and CO2
#Filter the data as three separate dfs containing 2010, 2016, 2023: Represented Test Veh Model, CO2, CO
CO2StatsByYear = cars.groupby('Model Year')['CO2 (g/mi)'].agg(['mean', 'median', 'std'])
COStatsByYear = cars.groupby('Model Year')['CO (g/mi)'].agg(['mean', 'median', 'std'])

# Pull general stats for each and either graph or print them out
print("CO2 Stats by Year: \n")
print( CO2StatsByYear, '\n')


print("CO Stats by Year: \n")
print( COStatsByYear)

/Users/chandlersmith/Desktop/CS6140/HW5
(11304, 42)
CO2 Stats by Year: 

             mean  median    std
Model Year                      
2010       348.61  332.08 117.80
2016       302.18  281.84 106.59
2023       314.77  295.00 115.06 

CO Stats by Year: 

            mean  median  std
Model Year                   
2010        0.35    0.21 0.43
2016        0.40    0.18 5.00
2023        0.33    0.19 0.52


Conclusion: When testing the claim from the EPA that pollutants' levels have generally decreased over time, the evidence is inconclusive when looking to back up that this statement is true. While there was a decrease from 2010 to 2023, the data suggest that the lowest year for pCO2 ollutants' levels was 2016. In 2016, the mean was 302 and the median was 281, which is substantially lower than 2023 numbers. That being said, it is true that the CO numbers were lowest in 2023, however, they went up from 2010 to 2016, providing evidence that contradicts the claim that pollutants' levels decreased over time. 

# Nearest Neighbor Vehicals

Using numerical features, choose 5 vehicals of your liking and find Ball Tree and KDtree nearest neighbors. 


In [226]:
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree

# Reload the data 
%cd '/Users/chandlersmith/Desktop/CS6140/HW5'
os.listdir()

# The first column is index: skipping that column
EPAcars = pd.read_csv("cars_10_16_23.csv")
#print(EPAcars.shape)

dropCols = ['Aftertreatment Device Desc','# of Cylinders and Rotors', 
                        'Aftertreatment Device Cd', 'THC (g/mi)', 'CH4 (g/mi)', 
                        'NOx (g/mi)', 'PM (g/mi)', 'N2O (g/mi)']
cars = EPAcars.drop(dropCols, axis=1)
cars = cars.dropna()
#print(cars)

## ----- COMPARISON BY YEAR, UNCOMMENT THE ONE YOU WANT TO COMPARE
#cars = cars.loc[cars['Model Year'] == 2010]
#cars = cars.loc[cars['Model Year'] == 2016]
#cars = cars.loc[cars['Model Year'] == 2023]


cars = cars.drop_duplicates(subset='Represented Test Veh Model', keep='first')

print(cars)
#standardize all numerical (except for the response)
for i in cars:
    if cars[i].dtypes == 'number': # and i != 'CO (g/mi)'
        cars[i] = StandardScaler.fit_transform(cars[i])

#Encode all categorical
for i in cars:
    if cars[i].dtype == "object" and i != 'Represented Test Veh Model':
        cars = cars.drop([i], axis=1)


# Choose the features to use for predicting nearest neighbors
#print(cars)
features = ['Rated Horsepower', 'RND_ADJ_FE', 'Axle Ratio', 'CO (g/mi)' ]

# Create a new dataframe with the chosen features
X = cars[features]


/Users/chandlersmith/Desktop/CS6140/HW5
       Unnamed: 0 Represented Test Veh Make Transmission Overdrive Desc  \
0               0              Aston Martin          Top gear ratio < 1   
2               2              Aston Martin          Top gear ratio < 1   
4               4              Aston Martin          Top gear ratio < 1   
6               6              Aston Martin          Top gear ratio < 1   
8               8              Aston Martin          Top gear ratio < 1   
...           ...                       ...                         ...   
11278        4624                     Volvo         Top gear ration < 1   
11280        4626                     Volvo         Top gear ration < 1   
11283        4629                     Volvo         Top gear ration < 1   
11286        4632                     Volvo         Top gear ration < 1   
11294        4640                     Volvo         Top gear ration < 1   

       Equivalent Test Weight (lbs.) Represented Test Veh M

In [227]:
# Car 1
# Use the chosen features to build a BallTree
ball = BallTree(X)

# Extract the data for a single car model
car_model = 'X7 xDrive40i'
car_data = cars[cars['Represented Test Veh Model'] == car_model][features].dropna().values

# Use the car data to query the BallTree for the nearest neighbors
k = 5
dist, ind = ball.query(car_data.reshape(1, -1), k=k)

# Print the indices and distances of the nearest neighbors
ball = BallTree(X, leaf_size=2)              
print(f"Ball tree: the nearest indices to {car_model} are: {ind}, \nAnd the respective distances are: {dist}")

print("\n\n")

kd = KDTree(X, leaf_size=2)
dist_kd, ind_kd = kd.query(X[1:2], k=5)                             
print(f''' KD Tree: the nearest indices to {car_model} are: {ind_kd}, \nAnd the respective distances are: 
      {dist_kd}''')


Ball tree: the nearest indices to X7 xDrive40i are: [[  56  383 1400  758 1247]], 
And the respective distances are: [[0.         2.98895616 3.01772522 4.92617408 5.35862856]]



 KD Tree: the nearest indices to X7 xDrive40i are: [[   1  100    3 1077  101]], 
And the respective distances are: 
      [[0.         4.37556    8.09027138 8.21541197 9.51994522]]


In [228]:
# Car 2
# Use the chosen features to build a BallTree
ball = BallTree(X)

# Extract the data for a single car model
car_model = 'XF AWD'
car_data = cars[cars['Represented Test Veh Model'] == car_model][features].dropna().values

# Use the car data to query the BallTree for the nearest neighbors
k = 5
dist, ind = ball.query(car_data.reshape(1, -1), k=k)

# Print the indices and distances of the nearest neighbors
ball = BallTree(X, leaf_size=2)              
print(f"Ball tree: the nearest indices to {car_model} are: {ind}, \nAnd the respective distances are: {dist}")

print("\n\n")

kd = KDTree(X, leaf_size=2)
dist_kd, ind_kd = kd.query(X[1:2], k=5)                             
print(f''' KD Tree: the nearest indices to {car_model} are: {ind_kd}, \nAnd the respective distances are: 
      {dist_kd}''')


Ball tree: the nearest indices to XF AWD are: [[1248  219 1403 1251 1249]], 
And the respective distances are: [[0.         0.59287938 1.01001205 2.21560307 3.96253803]]



 KD Tree: the nearest indices to XF AWD are: [[   1  100    3 1077  101]], 
And the respective distances are: 
      [[0.         4.37556    8.09027138 8.21541197 9.51994522]]


In [229]:
# Car 3
# Use the chosen features to build a BallTree
ball = BallTree(X)

# Extract the data for a single car model
car_model = 'Velar MHEV'
car_data = cars[cars['Represented Test Veh Model'] == car_model][features].dropna().values

# Use the car data to query the BallTree for the nearest neighbors
k = 5
dist, ind = ball.query(car_data.reshape(1, -1), k=k)

# Print the indices and distances of the nearest neighbors
ball = BallTree(X, leaf_size=2)              
print(f"Ball tree: the nearest indices to {car_model} are: {ind}, \nAnd the respective distances are: {dist}")

print("\n\n")

kd = KDTree(X, leaf_size=2)
dist_kd, ind_kd = kd.query(X[1:2], k=5)                             
print(f''' KD Tree: the nearest indices to {car_model} are: {ind_kd}, \nAnd the respective distances are: 
      {dist_kd}''')


Ball tree: the nearest indices to Velar MHEV are: [[ 237 1391 1392 1386 1387]], 
And the respective distances are: [[0.         0.38232871 0.40674198 0.51100391 0.62981745]]



 KD Tree: the nearest indices to Velar MHEV are: [[   1  100    3 1077  101]], 
And the respective distances are: 
      [[0.         4.37556    8.09027138 8.21541197 9.51994522]]


In [230]:
# Car 4
# Use the chosen features to build a BallTree
ball = BallTree(X)

# Extract the data for a single car model
car_model = 'MAZDA3'
car_data = cars[cars['Represented Test Veh Model'] == car_model][features].dropna().values

# Use the car data to query the BallTree for the nearest neighbors
k = 5
dist, ind = ball.query(car_data.reshape(1, -1), k=k)

# Print the indices and distances of the nearest neighbors
ball = BallTree(X, leaf_size=2)              
print(f"Ball tree: the nearest indices to {car_model} are: {ind}, \nAnd the respective distances are: {dist}")

print("\n\n")

kd = KDTree(X, leaf_size=2)
dist_kd, ind_kd = kd.query(X[1:2], k=5)                             
print(f''' KD Tree: the nearest indices to {car_model} are: {ind_kd}, \nAnd the respective distances are: 
      {dist_kd}''')


Ball tree: the nearest indices to MAZDA3 are: [[268 400   7 266   6]], 
And the respective distances are: [[0.         1.16666866 1.22011529 1.82499834 1.9097934 ]]



 KD Tree: the nearest indices to MAZDA3 are: [[   1  100    3 1077  101]], 
And the respective distances are: 
      [[0.         4.37556    8.09027138 8.21541197 9.51994522]]


In [231]:
# Car 5
# Use the chosen features to build a BallTree
ball = BallTree(X)

# Extract the data for a single car model
car_model = 'Artura'
car_data = cars[cars['Represented Test Veh Model'] == car_model][features].dropna().values

# Use the car data to query the BallTree for the nearest neighbors
k = 5
dist, ind = ball.query(car_data.reshape(1, -1), k=k)

# Print the indices and distances of the nearest neighbors
ball = BallTree(X, leaf_size=2)              
print(f"Ball tree: the nearest indices to {car_model} are: {ind}, \nAnd the respective distances are: {dist}")

print("\n\n")

kd = KDTree(X, leaf_size=2)
dist_kd, ind_kd = kd.query(X[1:2], k=5)                             
print(f''' KD Tree: the nearest indices to {car_model} are: {ind_kd}, \nAnd the respective distances are: 
      {dist_kd}''')


Ball tree: the nearest indices to Artura are: [[ 269 1304 1322 1303  221]], 
And the respective distances are: [[0.         1.04565643 1.10137657 1.10162917 3.41747193]]



 KD Tree: the nearest indices to Artura are: [[   1  100    3 1077  101]], 
And the respective distances are: 
      [[0.         4.37556    8.09027138 8.21541197 9.51994522]]


Does NN mee intuative expectations? 
It was a bit surprising to me! Some of the models I wouldn't have thought were good matches turned out to be very close when looking at the specific features! With some hand testing I found these car's varied a bit year to year, mostly due to car version availability with nearest neighbors. 

What is the overall conclusion?
Given the variety of techniques above, which one to use will likely be contextual and require testing. In generally, it is proabbly going to be a strong method to test several methods and use the one that produces the best model. 